In [1]:
import numpy as np
from PIL import Image
import os

In [3]:
os.getcwd()
os.listdir()

with Image.open("src_imgs/4.1.08.tiff") as f:
  w = f.width
  h = f.height
  rarr =[[0]*h for i in range(w)]
  garr = [[0]*h for i in range(w)]
  barr = [[0]*h for i in range(w)]
  for i in range(w):
    for j in range(h):
      pixels = f.getpixel((i,j))
      rarr[i][j] = pixels[0]
      garr[i][j] = pixels[1]
      barr[i][j] = pixels[2]

In [4]:
rArrMatr = np.asmatrix(rarr)
svdRarrP,svdRarrS, svdRarrQT = np.linalg.svd(rArrMatr)

In [5]:
print(svdRarrS)

[4.50254979e+04 4.05135633e+03 3.57266299e+03 3.22067443e+03
 2.38338014e+03 2.25400670e+03 2.16090474e+03 1.92085038e+03
 1.85075687e+03 1.49697694e+03 1.30390754e+03 1.09948044e+03
 1.01062607e+03 8.83351269e+02 8.07541447e+02 7.89915068e+02
 6.96930769e+02 6.70930792e+02 6.05936985e+02 5.97144460e+02
 5.75254658e+02 5.33605668e+02 5.20639195e+02 4.32124644e+02
 4.15541314e+02 3.98098652e+02 3.74139890e+02 3.50242917e+02
 3.48366001e+02 3.32074963e+02 3.05498099e+02 2.96713528e+02
 2.69955570e+02 2.59365610e+02 2.50000731e+02 2.39090692e+02
 2.23134719e+02 2.12438651e+02 2.01055619e+02 1.87443566e+02
 1.79195795e+02 1.70707764e+02 1.65056329e+02 1.58542227e+02
 1.50034208e+02 1.42002083e+02 1.38490894e+02 1.34985168e+02
 1.27729367e+02 1.20067681e+02 1.15689962e+02 1.09109888e+02
 1.06868525e+02 9.86033288e+01 9.57365556e+01 9.24487708e+01
 9.12912605e+01 8.84073826e+01 8.41419554e+01 8.02302722e+01
 7.91532733e+01 7.47462475e+01 7.40677216e+01 6.96839558e+01
 6.52633721e+01 6.339112

In [ ]:
#Ideally we have 3 test cases 

#Count clk cycle per second test just put code inbetween
start = count()

#Do shit here 
sleep(1)
elapsed = count_end() - start
print(f'Elapsed cycles: {elapsed:,}')

#Image print size
def CheckSize(fpatharr):
    for i in fpatharr:
        fsize = os.path.getsize(i)
        print("============\n Image size for: "+ i +"\n  
        Bit Size:"+fsize+"\n")

#Verify image composition post compression:
#Ideally have some code to check pixel by pixel rgbA vals of the image








In [6]:
import numpy as np
import matplotlib.pyplot as plt

def decompose_image(image):
    # Convert the image to a numpy array
    image_array = np.array(image)
    height, width, _ = image_array.shape
    image_matrix = image_array.reshape(height * width, 3)
    U, S, V = np.linalg.svd(image_matrix)

    red_channel = np.dot(U[:, 0] * S[0], V[0, :])
    green_channel = np.dot(U[:, 1] * S[1], V[1, :])
    blue_channel = np.dot(U[:, 2] * S[2], V[2, :])

    red_channel = red_channel.reshape(height, width)
    green_channel = green_channel.reshape(height, width)
    blue_channel = blue_channel.reshape(height, width)
    decomposed_image = np.dstack((red_channel, green_channel, blue_channel))

    return decomposed_image

# Load an image
image = plt.imread('src_imgs/4.1.08.tiff')

# Decompose the image into color channels
decomposed_image = decompose_image(image)

# Display the original image and decomposed channels
plt.subplot(2, 2, 1)
plt.imshow(image)
plt.title('Original Image')

plt.subplot(2, 2, 2)
plt.imshow(decomposed_image)
plt.title('Decomposed Image')

plt.subplot(2, 2, 3)
plt.imshow(decomposed_image[:, :, 0], cmap='gray')
plt.title('Red Channel')

plt.subplot(2, 2, 4)
plt.imshow(decomposed_image[:, :, 1], cmap='gray')
plt.title('Green Channel')

plt.show()


: 

: 